In [3]:
import pandas as pd # Pour manipuler les données
import re # Pour extraire des schémas

In [4]:
### Charger les données ###
chiffre_clef = pd.read_excel('data/chiffre_clef.xlsx')
actif = pd.read_excel('data/actif.xlsx')
passif = pd.read_excel('data/passif.xlsx' , header=1)

In [5]:
### Nettoyage des données ###
def clean_dates(df : pd.DataFrame)  -> None:
    '''
    Extrait les colonnes des dates , les transforme en objet TimeStamps , lisible en tant que dates par pandas.
    La modification se fait inplace (cette fonction ne retourne pas un dataframe , elle le modifie simplement)
    '''

    to_clean = list(df.columns)
    pattern = re.compile('\\d{2}/\\d{2}/\\d{4}') # Schéma de regex permettant d'extraire les dates
    
    for i in range(len(to_clean)):
        col = to_clean[i]
        result = re.findall(pattern=pattern , string=col)
        if len(result) == 1:
            to_clean[i] = pd.to_datetime(result[0] , format="%d/%m/%Y")

    df.columns = to_clean


def reverse_column_and_index(df : pd.DataFrame) -> pd.DataFrame:
    ''' 
    Initialement , les dates font office de colonnes. Il serait plus efficient d'inverser avec l'index , qui représente des catégories 
    pour afficher correctement les valeurs en graphiques
    '''
    df.index = df['Comptes sociaux']
    df = df.drop('Comptes sociaux' , axis=1)
    df = df.T # Inverser les colonnes et l'index
    return df


def turn_into_float_and_clean(df: pd.DataFrame) -> pd.DataFrame:
    ''' 
    Actuellement , toute les colonnes de nos jeux de donnée sont au format object. Il faut donc les transformer à un format numérique. 
    Il faut aussi se débarasser des espaces , caractères spéciaux , ect ... 
    '''
    df = df.applymap(lambda x: str(x).replace('\xa0', '').strip() if pd.notnull(x) else x)
    df = df.replace('' , '0')
    df = df.astype(float)
    df = df.drop([col for col in df.columns if col.strip() == ''], axis=1)
    df = df.drop(df.index[0])
    df = df.reset_index() # Pour accéder aux dates avec plotly
    df = df.rename(columns={'index': 'date'})
    df.columns = df.columns.str.replace('\xa0', '')
    df['year'] = df['date'].dt.year

    for col in df.drop('date' , axis=1):
        if df[col].sum() == 0.0:
            df = df.drop(col , axis=1)

    return df
    

# Extraire les dates
clean_dates(chiffre_clef)
clean_dates(actif)
clean_dates(passif)

# Transposer en série temporelle 
chiffre_clef = reverse_column_and_index(chiffre_clef)
actif = reverse_column_and_index(actif)
passif = reverse_column_and_index(passif)


# Transformer en type integers et nettoyer les colonnes vides
passif = turn_into_float_and_clean(passif)
actif = turn_into_float_and_clean(actif)
chiffre_clef = turn_into_float_and_clean(chiffre_clef)


# Sauvegarder les jeux de données propres
passif.to_csv('data/passif.csv' , index=False)
actif.to_csv('data/actif.csv' , index=False)
chiffre_clef.to_csv('data/chiffre_clef.csv' , index=False)